In [1]:
import os
from dotenv import load_dotenv
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
load_dotenv()

e:\Codes\Data Sciene\AI\Real_Estate_Chatbot\.venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = 'realestatebot'
namespace = 'docs-weblinks'

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import Settings, VectorStoreIndex, get_response_synthesizer  # noqa: F401
from llama_index.core.chat_engine.context import ContextChatEngine
from llama_index.core.llms import ChatMessage
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.postprocessor import LongContextReorder
from llama_index.core.retrievers import VectorIndexRetriever
# from llama_index.llms.gemini import Gemini
from llama_index.llms.groq import Groq

In [4]:
from llama_index.core.query_engine import RetrieverQueryEngine
from crewai import Agent, Task, Crew, LLM
from crewai_tools import LlamaIndexTool

In [5]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [6]:
Settings.embed_model=embed_model
Settings.llm = Groq(api_key=os.environ.get("GROQ_API_KEY"), model = 'llama3-70b-8192')

In [12]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

In [24]:
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """ You are an assistant for answering real estate-related queries. Use the following pieces of retrieved information to answer questions about properties, such as available listings, property details (e.g., number of BHKs, facilities), and other relevant information. Provide clear and direct answers as if you're responding to a user's inquiry, without mentioning any documents or contexts. If you don't know the answer, just say that you don't know.

You should always answer using the provided context and only the context from the document."""
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [37]:
def pinecone_vectorstore():
    index = pc.Index(index_name)
    pinecone_vector_store = PineconeVectorStore(pinecone_index=index, namespace='docs-weblinks')

    return VectorStoreIndex.from_vector_store(vector_store=pinecone_vector_store)

reorder = LongContextReorder()
vector_index_retriever = VectorIndexRetriever(
index=pinecone_vectorstore(),
namespace=namespace,
postprocessors=[reorder],
similarity_top_k=7),

# index=pinecone_vectorstore()
# namespace=namespace,
response_synthesizer = get_response_synthesizer(response_mode='refine', text_qa_template=text_qa_template)
postprocessors = [reorder]

In [38]:
query_engine = RetrieverQueryEngine(
        retriever=vector_index_retriever[0],
        response_synthesizer=response_synthesizer,
)

In [41]:
res = query_engine.query("What are different properties sale at jaipur")

In [43]:
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Real Estate Bot",
    description="This tool is a real estate chatbot that helps answer property-related queries, including available listings, BHK details, facilities, and other relevant real estate information using context from the retrieved documents."
)

In [44]:
llm = LLM(
    model = 'groq/llama-3.1-70b-versatile',
    api_key=os.environ.get("GROQ_API_KEY")
)

In [45]:
# Define the Real Estate Researcher agent
real_estate_agent = Agent(
    role="Real Estate Information Provider",
    goal="Provide tailored responses and retrieve relevant information from the document according to the user's query: {input}",
    backstory="""An expert in finding and filtering relevant information from the provided documents or databases.
    Your goal is to give concise and accurate information based on the user's query.""",
    llm=llm,
    tools=[query_tool],  # Pass the real estate bot query tool
    verbose=True,
)

# Define the task for the agent
real_estate_task = Task(
    agent=real_estate_agent,
    description="""Your task is to filter and retrieve the most relevant information from the document based on the user's query: {input}.
    Never provide any information outside of the document. If you don't know the answer, simply say that you don't know. Never give incorrect information.""",
    expected_output="The output should be a single paragraph.",
    output_file='./output_crew_ai.txt'
)

# Create the Crew instance
my_crew = Crew(agents=[real_estate_agent], tasks=[real_estate_task])

In [49]:
crew = my_crew.kickoff(inputs={"input": "I want a flat in jaipur region in location shanti niketan"})

# Agent: Real Estate Information Provider
## Task: Your task is to filter and retrieve the most relevant information from the document based on the user's query: I want a flat in jaipur region in location shanti niketan.
    Never provide any information outside of the document. If you don't know the answer, simply say that you don't know. Never give incorrect information.


# Agent: Real Estate Information Provider
## Thought: Thought: I need to query the database to find relevant information about flats in the Jaipur region, specifically in the Shanti Niketan location.
## Using tool: Real Estate Bot
## Tool Input: 
"{\"query\": \"flats in Jaipur region in Shanti Niketan location\"}"
## Tool Output: 
I don't know. The provided context does not mention Shanti Niketan location.


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Real Estate Bot(*args: Any, **kwargs: Any) -> Any
Tool Description: Real Estate Bot(query: 'stri

2024-09-30 00:17:28,238 - 9976 - llm.py-llm:88 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j3nva787fhcrbfvsdrp1q2ps` on : Limit 500000, Used 499685, Requested 1087. Please try again in 2m13.293399999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Real Estate Information Provider
## Task: Your task is to filter and retrieve the most relevant information from the document based on the user's query: I want a flat in jaipur region in location shanti niketan.
    Never provide any information outside of the document. If you don't know the answer, simply say that you don't know. Never give incorrect information.


In [48]:
crew.raw

"Based on your query, I didn't find a 2BHK flat in Jaipur available below 30 lakhs. However, I'd like to bring to your attention a few options in Bhankrota, Jaipur, such as a 3 BHK flat with an area of 1800 Sq.ft, priced between 57-60 lakhs, which might be of interest to you. If you're open to exploring more options, I suggest contacting the builder or agent for more information about the prices of available 2BHK flats in Jaipur that fit your budget of below 30 lakhs."